# Load and observe data

Firstly, import necessary modules

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder


Read data and concat them into one df

In [41]:
df_train = pd.read_csv(
    'drive/My Drive/data/train.csv',
    #dtype={'Field_34': float}
)
df_test = pd.read_csv('drive/My Drive/data/test.csv')

df = pd.concat([df_train, df_test])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Get some information about data

In [6]:
df_train.describe()

,id,label,Field_3,Field_10,Field_13,Field_14,Field_16,Field_17,Field_19,Field_20,Field_21,Field_22,Field_23,Field_24,Field_26,Field_27,Field_28,Field_29,Field_30,Field_31,ngaySinh,namSinh,Field_37,Field_39,Field_41,Field_42,Field_50,Field_51,Field_52,Field_53,Field_57,Field_58,Field_59,Field_60,Field_63,Field_64,Field_67,Field_69,Field_70,Field_71,...,partner2_G,partner2_H,partner2_K,partner2_L,partner3_A,partner3_B,partner3_C,partner3_D,partner3_E,partner3_F,partner3_G,partner3_H,partner3_K,partner3_L,partner4_A,partner4_B,partner4_C,partner4_D,partner4_E,partner4_F,partner4_G,partner4_H,partner4_K,partner4_L,partner5_A,partner5_B,partner5_C,partner5_D,partner5_E,partner5_F,partner5_G,partner5_H,partner5_K,partner5_L,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
count,53030.000000,53030.000000,25564.000000,25564.000000,25564.0,25564.0,25564.0,25564.0,25564.000000,2.556400e+04,25564.000000,25564.000000,25564.000000,25564.0,25564.0,2.556400e+04,25564.000000,25564.000000,25564.0,25564.0,2.556400e+04,25564.000000,25564.0,25564.000000,25564.000000,25564.000000,25564.000000,25564.000000,25564.0,25564.000000,25564.0,25564.000000,25564.000000,25563.000000,25564.000000,24144.000000,2.556400e+04,24791.000000,25564.000000,25564.000000,...,48283.0,48283.000000,48283.0,48283.0,48283.000000,48283.0,48283.0,48283.000000,48283.000000,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.0,48283.000000,48283.000000,48283.0,48283.0,48283.000000,48283.000000,48283.000000,48283.000000,48283.0,48283.0,48283.0,50518.000000,36397.00000,27782.000000,36320.000000,35981.000000,53030.000000
mean,26514.500000,0.316726,1.538687,1.096542,1.0,1.0,1.0,1.0,0.267022,3.603181e+06,4.485331,0.999452,31.472540,1.0,1.0,2.252840e+05,10003.327257,6.548232,1.0,1.0,1.988407e+07,0.056173,1.0,0.809537,0.762205,0.748709,0.999765,0.999844,1.0,0.003364,1.0,1.166289,14.984431,13.416148,3.045024,2.118497,5.279420e+05,0.306730,0.418029,0.342623,...,0.0,0.030735,0.0,0.0,0.341880,0.0,0.0,0.210778,0.001263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.306464,0.254334,0.0,0.0,0.000124,0.000124,0.000124,0.000041,0.0,0.0,0.0,1.087236,20.80144,30.911307,-1.793163,-32.149412,1.668659
std,15308.586724,0.465204,0.498511,0.332593,0.0,0.0,0.0,0.0,0.835968,2.282034e+06,0.281697,0.053069,46.238001,0.0,0.0,8.423081e+05,37951.762216,6.352628,0.0,0.0,6.858198e+04,0.230599,0.0,0.392674,0.425741,0.433764,0.015319,0.012508,0.0,0.057904,0.0,2.004592,16.999536,18.152622,2.235615,1.539845,2.923003e+05,0.321212,0.281048,0.269400,...,0.0,0.172602,0.0,0.0,0.474345,0.0,0.0,0.407865,0.035522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461030,0.435491,0.0,0.0,0.011147,0.011147,0.011147,0.006436,0.0,0.0,0.0,0.297815,40.10993,35.968215,32.176041,63.442452,1.090910
min,0.000000,0.000000,1.000000,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.0,1.0,0.000000e+00,0.000000,0.000000,1.0,1.0,1.905062e+07,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.000000,3.000000e+00,0.000000,0.000000,-0.500000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,-267.00000,0.000000,-267.000000,-290.000000,1.000000
25%,13257.250000,0.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,0.000000,1.490000e+06,4.500000,1.000000,1.000000,1.0,1.0,0.000000e+00,0.000000,0.000000,1.0,1.0,1.985022e+07,0.000000,1.0,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,0.000000,1.0,0.000000,4.000000,2.000000,2.000000,1.000000,2.681360e+05,0.000000,0.120000,0.165000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,4.00000,5.326888,-5.600000,-69.000000,1.000000
50%,26514.500000,0.

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53030 entries, 0 to 53029
Columns: 195 entries, id to Field_82
dtypes: float64(133), int64(3), object(59)
memory usage: 78.9+ MB


In [8]:
df_train['label'].value_counts()

0    36234
1    16796
Name: label, dtype: int64

We can see that data is imbalance

# Pre-process data

In [39]:
string_fields = [
    'Field_12',
    'Field_18',
    'Field_46',
    'Field_47',
    'Field_48',
    'Field_49',
    'Field_61',
    'Field_62',
    'Field_65',
    'Field_66',
    'maCv',
    'data.basic_info.locale',
    'currentLocationCity',
    'currentLocationCountry',
    'currentLocationName',
    'currentLocationState',
    'currentLocationState',
    'homeTownCity',
    'homeTownCountry',
    'homeTownName',
    'homeTownState',
    'brief'
]

In [10]:
def getNullCounts(df_temp):
    null_counts = pd.DataFrame(df_temp.isnull().sum(axis = 0))
    null_counts.reset_index(inplace = True)
    null_counts.columns = ['Field', 'Number of null']
    null_counts['% null'] = null_counts['Number of null'] / len(list(df_temp.index))
    null_counts.sort_values(
        ['Number of null'], 
        ascending = False,
        axis = 0,
        inplace = True
    )
    return null_counts

def getBadFields(df_temp):
    null_counts = getNullCounts(df_temp)
    bad_fields = null_counts['Field'][null_counts['% null'] > 0.7]
    return list(bad_fields)

def dropBadFields(df_temp):
    bad_fields = getBadFields(df_temp)
    bad_fields = [item for item in bad_fields if item not in string_fields]
    return df_temp.drop(
        bad_fields,
        axis = 1,
    )
    

In [40]:
def dropObjectFields(df_temp):
    object_fields = df_temp.loc[:, list(df_temp.dtypes == object)].columns
    object_fields = list(object_fields)
    object_fields = [item for item in object_fields if item not in string_fields]
    return df_temp.drop(
        object_fields,
        axis = 1,
    )
def encodeFields(df_temp):
  for field in string_fields:
    df_temp[field] = df_temp[field].fillna('nan')
    le = LabelEncoder()
    le.fit(list(df_temp[field].unique()))
    df_temp[field] = le.transform(df_temp[field])
  return df_temp



In [42]:
def preProcess(df_temp):
    df_temp = dropBadFields(df_temp)
    df_temp = dropObjectFields(df_temp)
    df_temp = encodeFields(df_temp)
    
    df_temp = df_temp.fillna(0)
    
    return df_temp

In [43]:
df = preProcess(df)

In [44]:
df

,id,label,Field_3,Field_10,Field_12,Field_13,Field_14,Field_16,Field_17,Field_18,Field_19,Field_20,Field_21,Field_22,Field_23,Field_24,Field_26,Field_27,Field_28,Field_29,Field_30,Field_31,ngaySinh,namSinh,Field_37,Field_39,Field_41,Field_42,Field_46,Field_47,Field_48,Field_49,Field_50,Field_51,Field_52,Field_53,Field_57,Field_58,Field_59,Field_60,...,partner2_H,partner2_K,partner2_L,partner3_A,partner3_B,partner3_C,partner3_D,partner3_E,partner3_F,partner3_G,partner3_H,partner3_K,partner3_L,partner4_A,partner4_B,partner4_C,partner4_D,partner4_E,partner4_F,partner4_G,partner4_H,partner4_K,partner4_L,partner5_A,partner5_B,partner5_C,partner5_D,partner5_E,partner5_F,partner5_G,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,0,1.0,1.0,1.0,4,1.0,1.0,1.0,1.0,7754,0.000,4258600.0,4.5,1.0,16.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,19910615.0,0.0,1.0,1.0,1.0,1.0,4397,0,12452,12153,1.0,1.0,1.0,0.0,1.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,1.0,0.0,0.000000,0.000000,0.0,1
1,1,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5,14283,14436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1.0,0.0,0.000000,0.000000,0.0,1
2,2,0.0,2.0,1.0,7,1.0,1.0,1.0,1.0,8522,0.000,5000000.0,4.5,1.0,10.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,19980601.0,0.0,1.0,1.0,1.0,1.0,1857,4,12574,12694,1.0,1.0,1.0,0.0,1.0,0.0,5.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,2
3,3,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5,14283,14436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,3
4,4,1.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5,14283,14436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5,14283,14436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,24.0,42.644640,-10.722222,-160.0,3
20377,73407,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5,14283,14436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1.0,44.0,13.740300,6.562500,0.0,3
20378,73408,0.0,1.0,1.0,7,1.0,1.0,1.0,1.0,8522,4.136,6162640.0,4.5,1.0,172.0,1.0,1.0,0.0,0.0,24.0,1.0,1.0,19820418.0,0.0,1.0,1.0,1.0,1.0,15642,3,14939,15158,1.0,1.0,1.0,0.0,1.0,0.0,6.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,1.0,0.0,0.000000,0.000000,0.0,1
20379,73409,0.0,0.0,0.0,7,0.0,0.0,0.0,0.0,8522,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17055,5

# Create and train model

In [45]:
df_train = df[df['id'] < 53030]
df_test = df[df['id'] >= 53030]

df_train = df_train.sample(frac = 1, random_state = 1).reset_index(drop = True)

In [46]:
X_train = df_train.drop(columns = ['label', 'id'])
Y_train = df_train['label']
X_test = df_test.drop(columns = ['label', 'id'])
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(53030, 155)
(53030,)
(20381, 155)


In [47]:
creditTree = DecisionTreeClassifier()
parameters = {'max_depth': (10, 20 ,30 ,40, 50, 60, 70, 80, 90, 100),
              'criterion': ('gini', 'entropy')
             }
model = GridSearchCV(estimator = creditTree, 
                     param_grid = parameters
                    )
model.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ('gini', 'entropy'),
                

In [50]:
Y_test = model.predict_proba(X_test)[:, 1]
Y_test
print(model.score(X_train, Y_train))
print(model.best_estimator_)

0.7565151800867433
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [55]:
res_df = pd.DataFrame({'id': df_test['id'], 'label': Y_test})
res_df.to_csv('submission.csv', index=False)

## Some code to double check pre-processing

In [ ]:
print(lr.classes_)
df_train.to_csv('test.csv', index = False)
print(dropObjectFields(df_train).info())
df_train = dropBadFields(df_train)
print(getNullCounts(df_train))